In [2]:
from fuzzy_ngram_matrix_factors_model import SentimentModel

In [5]:
heavy_reg_model_v1 = SentimentModel.load_from_checkpoint('lightning_logs/heavy_regularization/version_1/checkpoints/last.ckpt', no_load_glove=False)
heavy_reg_model_v1.eval()

SentimentModel(
  (embedding): Embedding(1500000, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
    (3): Conv2d(1, 100, kernel_size=(7, 300), stride=(1, 1))
  )
  (user_embedding): Embedding(123356, 10)
  (product_embedding): Embedding(49945, 10)
  (user_product_dim_reduction): Identity()
  (user_product_pool): Identity()
  (feature_weights): Linear(in_features=822, out_features=5, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [6]:
from preprocess_data import load_data

train_df, test_df = load_data()

Loading preprocessed Parquet files...... Done!


In [11]:
train_df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Summary_glove_tokens,Text_glove_tokens,Summary_glove_tokens_np,Text_glove_tokens_np,HelpfulnessRatio,LogHelpfulnessDenominator
0,914403,B0009W5KHM,AV6QDP8Q0ONK4,2,2,1341014400,GOOD FUN FILM,While most straight to DVD films are not worth...,5.0,"[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ...","[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ...",1.000000,1.098612
1,354887,6303079709,A2I8RXJN80A2D2,0,0,1168819200,Movie Review,"I have wanted this one for sometime, also. I ...",5.0,"[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777...","[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777...",0.000000,0.000000
2,1407653,B004H0M2XC,A3FHV3RV8Z12E6,0,0,1386201600,When is it a good time to Consent?,Actually this was a pretty darn good indie fil...,4.0,"[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21...","[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21...",0.000000,0.000000
3,1377458,B003ZJ9536,A12VLTA3ZHVPUY,1,1,1348704000,TRUTH,Episodes 37 to 72 of the series press on in a ...,5.0,[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ...",[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ...",1.000000,0.693147
4,475323,630574453X,A13NM1PES9OXVN,2,3,970012800,Intelligent and bittersweet -- stays with you,"I was really impressed with this movie, but wa...",3.0,"[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,...","[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,...",0.666667,1.386294


In [9]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
from data_processor import DataProcessor

data_processor = DataProcessor(no_unknowns=False, unknown_threshold=cull_unknown_threshold)
data_processor.fit_encoders(train_df)

ValueError: too many dimensions 'str'